In [1]:
import os
import json
import numpy as np

with open('../../brain-knowledge/corpus.json', 'r') as f:
    corpus = json.load(f)

with open('node_benchmark.json', 'r') as f:
    node_benchmark = json.load(f)

with open('../../brain-surface/annotation/aparc_region_names.txt', 'r') as f:
    region_names = f.read().split('\n')

data = {}
for region in region_names:
    item = {
        'name': region,
        'synonyms': corpus[region]['Synonym'],
        'description': corpus[region]['Description'],
    }
    relative_corpus = ''
    for key, value in corpus[region]['Relational corpus'].items():
        relative_corpus += f' {value}\n'
    item['related corpus'] = relative_corpus
    other_name = ''.join(region.split(' '))
    other_name = other_name[0].lower() + other_name[1:]
    benchmark = {}
    for key, value in node_benchmark['left ' + other_name].items():
        histogram = np.histogram(value, bins=int(np.sqrt(len(value))))
        benchmark['left ' + other_name + ' ' + key] = {
            'hist': np.around(histogram[0], 2).tolist(),
            'bins': np.around(histogram[1][0: -1], 2).tolist()
        }
    for key, value in node_benchmark['right ' + other_name].items():
        histogram = np.histogram(value, bins=int(np.sqrt(len(value))))
        benchmark['right ' + other_name + ' ' + key] = {
            'hist': np.around(histogram[0], 2).tolist(),
            'bins': np.around(histogram[1][0: -1], 2).tolist()
        }
    item['benchmark'] = benchmark
    data[other_name] = item

with open('../../brain-knowledge/aparc_region_detail.json', 'w') as f:
    json.dump(data, f, indent=4)